In [13]:
from keras.applications import xception
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Input
from keras.models import Model, load_model
from keras.optimizers import SGD
from keras.applications.xception import preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array

import numpy as np
import os
import pickle
import h5py

from classes import CLASSES, LABELS_INDEX

In [2]:
image_size = (299, 299)
base_model = xception.Xception(include_top=False, input_shape=(*image_size, 3), weights='imagenet', pooling='avg')

In [ ]:
def reorganize_data(label_file='./dataset/labels.csv',
                    base_dir='./dataset/train/',
                    train_dir=None,
                    validation_dir=None, validation_split=0.1, ext='.jpg'):
    # Read label file to get list of file name and label
    data_by_class = {}
    with open(label_file, 'r') as f:
        next(f)
        for line in f:
            filename, label = line.split(',')
            filename = filename.strip()
            label = label.strip()
            if label not in data_by_class:
                data_by_class[label] = []
            data_by_class[label].append(filename)
    # Create dirs
    validation_split = 0.1
    train_dir = train_dir or os.path.join(base_dir, 'train')
    validation_dir = validation_dir or os.path.join(base_dir, 'validation')
    os.mkdir(train_dir)
    os.mkdir(validation_dir)
    # Moving everything into its folder
    for label, files in data_by_class.items():
        label_dir_train = os.path.join(train_dir, label)
        os.mkdir(label_dir_train)
        label_dir_val = os.path.join(validation_dir, label)
        os.mkdir(label_dir_val)

        val_index = len(files) * validation_split
        for i, filename in enumerate(files):
            if ext:
                filename += ext
            old_path = os.path.join(base_dir, filename)
            if os.path.isfile(old_path):
                if (i + 1) < val_index:
                    os.rename(old_path, os.path.join(label_dir_val, filename))
                else:
                    os.rename(old_path, os.path.join(label_dir_train, filename))

In [14]:
# Freeze bottom layer
for layer in base_model.layers:
    layer.trainable = False

def get_top_model(base_model):
    top_input = Input(shape=base_model.output_shape[1:])
    x = Dense(1024, activation="relu", name='dense1024')(top_input)
    x = Dropout(0.5, name='drop05')(x)
    x = Dense(512, activation="relu", name='dense512')(x)
    x = Dropout(0.2, name='drop02')(x)
    output = Dense(len(CLASSES), activation="softmax", name='final')(x)
    model = Model(input=top_input, output=output)
    model.compile(
        loss="categorical_crossentropy",
        optimizer='adam',
        metrics=["accuracy"])
    return model

def get_final_model(base_model, top_model):
    final_output = top_model(base_model.output)
    final_model = Model(input=base_model.input, output=final_output)
    final_model.compile(
        loss='categorical_crossentropy',
        optimizer=SGD(lr=5e-5, momentum=0.9),
        metrics=["accuracy"]
    )
    return final_model


In [4]:
def predict(model, path):
    img = load_img(path, target_size=(299, 299))
    img_arr = img_to_array(img)
    inp = preprocess_input(img_arr)
    inp = np.expand_dims(inp, axis=0)
    preds = model.predict(inp)
    return preds[0]

In [5]:
train_datagen =  ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    './dataset/images/train/', 
    target_size=image_size,
    batch_size=25,
    class_mode='categorical', shuffle=False)
val_generator = val_datagen.flow_from_directory(
    './dataset/images/validation//', 
    target_size=image_size,
    batch_size=25,
    class_mode='categorical', shuffle=False)

Found 9357 images belonging to 122 classes.
Found 1015 images belonging to 122 classes.


In [ ]:
bottleneck_features_train = base_model.predict_generator(train_generator)

In [ ]:
hf = h5py.File('bottleneck_features-xcetp.h5', 'w')
hf.create_dataset('bottleneck_features_train', data=bottleneck_features_train)

In [ ]:
bottleneck_features_val = base_model.predict_generator(val_generator)
hf.create_dataset('bottleneck_features_val', data=bottleneck_features_val)
hf.close()

In [6]:
hf = h5py.File('bottleneck_features-xcetp.h5', 'r')
bottleneck_features_train = hf.get('bottleneck_features_train').value
bottleneck_features_val = hf.get('bottleneck_features_val').value

In [ ]:
bottleneck_features_train == bottleneck_features_train2

In [7]:
indices = np.array(train_generator.classes)
Y_train = np.zeros((len(train_generator.classes), len(train_generator.class_indices)))
Y_train[np.arange(len(train_generator.classes)), indices] = 1
indices = np.array(val_generator.classes)
Y_val = np.zeros((len(val_generator.classes), len(train_generator.class_indices)))
Y_val[np.arange(len(val_generator.classes)), indices] = 1

In [8]:
top_model = get_top_model(base_model)

/Users/nhutrinh/dogbreeed/env/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fi...)`
  if sys.path[0] == '':


In [9]:
history = top_model.fit(bottleneck_features_train, Y_train, batch_size=50, epochs=40)

Epoch 1/40
9357/9357 [==============================] - 5s 558us/step - loss: 1.3407 - acc: 0.6846
Epoch 2/40
9357/9357 [==============================] - 4s 469us/step - loss: 0.5153 - acc: 0.8483
Epoch 3/40
9357/9357 [==============================] - 4s 480us/step - loss: 0.4377 - acc: 0.8689
Epoch 4/40
9357/9357 [==============================] - 5s 492us/step - loss: 0.3638 - acc: 0.8838
Epoch 5/40
9357/9357 [==============================] - 4s 475us/step - loss: 0.3387 - acc: 0.8923
Epoch 6/40
9357/9357 [==============================] - 5s 496us/step - loss: 0.2897 - acc: 0.9084
Epoch 7/40
9357/9357 [==============================] - 5s 507us/step - loss: 0.2603 - acc: 0.9113
Epoch 8/40
9357/9357 [==============================] - 5s 518us/step - loss: 0.2477 - acc: 0.9200
Epoch 9/40
9357/9357 [==============================] - 5s 528us/step - loss: 0.2332 - acc: 0.9232
Epoch 10/40
9357/9357 [==============================] - 5s 555us/step - loss: 0.2220 - acc: 0.9258
Epoch 11/

In [10]:
top_model.evaluate(bottleneck_features_val, Y_val)

1015/1015 [==============================] - 1s 569us/step


[0.5873121570043376, 0.8817733994845687]

In [11]:
top_model.save_weights('top-xcep.w')

In [15]:
final_model = get_final_model(base_model, top_model)

/Users/nhutrinh/dogbreeed/env/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("mo...)`


In [ ]:
label_by_index = {v:k for k,v in train_generator.class_indices.items()}
CLASSES = [label_by_index[i] for i in label_by_index]

In [16]:
final_model.save('dogbreed-xcept.h5')
# top_model = load_model('./dogbreed.h5')

In [4]:
model = load_model('dogbreed-xcept.h5')

In [ ]:
top_model.fit_generator(generator=train_generator, epochs=2, shuffle=True)

In [17]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(final_model, './js_model_xcept')

In [ ]:
from keras.utils import CustomObjectScope
with CustomObjectScope({'relu6': mobilenet.relu6,'DepthwiseConv2D': mobilenet.DepthwiseConv2D}):
    model = load_model('dogbreed-mobilenet.h5')

In [6]:
def predict_label(model, path):
    p = predict(model, path)
    top_indices = p.argsort()[-5:][::-1]
    result = [(CLASSES[i], p[i],) for i in top_indices]
    result.sort(key=lambda x: x[1], reverse=True)
    return result


In [7]:
predict_label(model, './dataset/images/validation/sussex_spaniel/181e91cb6caf6739478d06231faa053d.jpg')

[('sussex_spaniel', 1.0),
 ('clumber', 4.001788e-12),
 ('bloodhound', 1.1285548e-14),
 ('cocker_spaniel', 4.84357e-15),
 ('french_bulldog', 2.0436948e-16)]

In [12]:
# Have to unfreeze layer before save to js otherwise not work
for layer in model.layers:
    layer.trainable = True

In [10]:
model.save_weights('test-xcetp.wh')

In [11]:
model.load_weights('test-xcetp.wh')

In [ ]:
model.add_weight

In [13]:
from keras.utils import CustomObjectScope
import tensorflowjs as tfjs
with CustomObjectScope():
    tfjs.converters.save_keras_model(model, './js_model_xcept')

In [ ]:
import keras as K
import tensorflow as tf
sess = K.backend.get_session()

In [ ]:
[CLASSES[i] for i in np.argsort(predict(model, './cuteness1.jpg'))[::-1][:5]]

In [ ]:
model.build(input_shape=base_model.input_shape)